In [20]:
%matplotlib inline

import numpy as np
import random
import pandas as pd
import requests as rq
import sys
import io
import os
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Dropout


%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Data Cleaning

In [2]:

##get data
##data is already from 2012-2020
data = pd.read_csv('data_file.csv')


##filter to only patriots

data = data.loc[(data['home_team'] == 'NE') | (data['away_team'] == 'NE')]
data.to_csv('cleaneddata.csv', index=False)

/var/folders/ys/6w09q22n4f73ds9jwmgbstjc0000gn/T/ipykernel_32235/1140940940.py:2: DtypeWarning: Columns (45,179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,213,214,218,219,220,222,224,226,233,234,235,236,237,238,248,249,253,254,255,260,262,263,266,267,268,269,283,284,294,295,301) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data_file.csv')


In [1]:


##preprocessing






In [3]:
# Get first 5 rows for testing
df = pd.read_csv("sam_data.csv")



In [6]:
head = df.head()
head


,play_id,game_id,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,half_seconds_remaining,...,rusher_player_name,play_type_nfl,roof,surface,temp,wind,home_coach,away_coach,game_stadium,success
0,53,2012_01_NE_TEN,REG,1,TEN,home,NE,TEN,80.0,1800.0,...,J.Locker,RUSH,outdoors,grass,74.0,4.0,Mike Munchak,Bill Belichick,LP Field,0.0
1,75,2012_01_NE_TEN,REG,1,TEN,home,NE,TEN,78.0,1773.0,...,C.Johnson,RUSH,outdoors,grass,74.0,4.0,Mike Munchak,Bill Belichick,LP Field,0.0
2,96,2012_01_NE_TEN,REG,1,TEN,home,NE,TEN,77.0,1731.0,...,NaN,PASS,outdoors,grass,74.0,4.0,Mike Munchak,Bill Belichick,LP Field,1.0
3,120,2012_01_NE_TEN,REG,1,TEN,home,NE,TEN,60.0,1692.0,...,C.Johnson,RUSH,outdoors,grass,74.0,4.0,Mike Munchak,Bill Belichick,LP Field,1.0
4,174,2012_01_NE_TEN,REG,1,TEN,home,NE,NE,46.0,1653.0,...,NaN,PASS,outdoors,grass,74.0,4.0,Mike Munchak,Bill Belichick,LP Field,0.0


In [5]:

head.to_csv("head.csv", index= False)

In [14]:
# Get rid of annoying messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [21]:

data = pd.read_csv('sam_data.csv')  


features = data[['down', 'yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind']]
target = data['play_type_nfl'].map({'PASS': 0, 'RUSH': 1})

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['yardline_100', 'half_seconds_remaining', 'game_seconds_remaining', 'temp', 'wind'])
    ]
)

X_processed = preprocessor.fit_transform(features)

X_lstm = X_processed.reshape((X_processed.shape[0], 1, X_processed.shape[1]))
y = target.values

X_train, X_test, y_train, y_test = train_test_split(X_lstm, y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(1, 5), return_sequences=True), 
    Dropout(0.2), 
    LSTM(32),  
    Dense(1, activation='sigmoid')  
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(X_train, y_train, epochs=50, validation_split=0.1, callbacks=[early_stopping], verbose=1)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {test_acc}')

predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int).reshape(-1)

for i in range(10):
    actual = 'PASS' if y_test[i] == 0 else 'RUSH'
    predicted = 'PASS' if predictions[i] == 0 else 'RUSH'
    print(f'Actual: {actual}, Predicted: {predicted}')


2024-04-10 23:11:09.979297: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 23:11:09.980736: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 23:11:09.981975: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/50


2024-04-10 23:11:10.379379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 23:11:10.380840: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 23:11:10.382128: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - ETA: 0s - loss: nan - accuracy: 0.5126

2024-04-10 23:11:21.006284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 23:11:21.007948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 23:11:21.009322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

526/526 [==============================] - 12s 17ms/step - loss: nan - accuracy: 0.5126 - val_loss: nan - val_accuracy: 0.5321
Epoch 2/50
526/526 [==============================] - 8s 15ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 3/50
526/526 [==============================] - 8s 14ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 4/50
526/526 [==============================] - 8s 14ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 5/50
526/526 [==============================] - 8s 14ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 6/50
526/526 [==============================] - 8s 15ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 7/50
526/526 [==============================] - 8s 15ms/step - loss: nan - accuracy: 0.5125 - val_loss: nan - val_accuracy: 0.5321
Epoch 8/50
526/526 [==============================]

2024-04-10 23:12:36.136543: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-04-10 23:12:36.137903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-04-10 23:12:36.139077: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

147/147 [==============================] - 1s 6ms/step
Actual: RUSH, Predicted: PASS
Actual: PASS, Predicted: PASS
Actual: RUSH, Predicted: PASS
Actual: RUSH, Predicted: PASS
Actual: RUSH, Predicted: PASS
Actual: RUSH, Predicted: PASS
Actual: PASS, Predicted: PASS
Actual: PASS, Predicted: PASS
Actual: RUSH, Predicted: PASS
Actual: PASS, Predicted: PASS
